# Portfolio

In [28]:
import pandas as pd
from beakerx import *

In [29]:
prices=pd.read_csv("data/price.csv", index_col=0, header=0, parse_dates=True)

In [30]:
SimpleTimePlot(prices, prices.keys())

In [32]:
from pyutil.portfolio.portfolio import Portfolio

# construct a 1/n portfolio, assets that come in later will initially not have any weight
def f(x):
    # how many assets are alive?
    n = x.notnull().sum()
    y = pd.Series(index=x.index)
    if n > 0:
        y[x.notnull()] = 1.0/n
    return y
    
p = Portfolio(prices, weights=prices.ffill().apply(f, axis=1))


In [34]:
print(dir(p))

['_Portfolio__before', '_Portfolio__f', '_Portfolio__prices', '_Portfolio__r', '_Portfolio__weights', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'apply', 'asset_returns', 'assets', 'cash', 'copy', 'empty', 'forward', 'index', 'iron_threshold', 'iron_time', 'leverage', 'nav', 'position', 'prices', 'sector_weights', 'sector_weights_final', 'snapshot', 'state', 'subportfolio', 'tail', 'top_flop_mtd', 'top_flop_ytd', 'trading_days', 'truncate', 'weight_current', 'weighted_returns', 'weights']


## Sector analysis

It is possible to assign each asset to a sector via sectormaps. 


In [55]:
print(p.assets)
sectormap = {"A": "S1", "B": "S1", "C": "S2", "D": "S2", "E": "S2", "F": "S2", "G": "S3"}
p.tail(5).sector_weights(sectormap)

['A', 'B', 'C', 'D', 'E', 'F', 'G']


,S1,S2,S3
2015-04-16,0.285714,0.571429,0.142857
2015-04-17,0.285714,0.571429,0.142857
2015-04-20,0.285714,0.571429,0.142857
2015-04-21,0.285714,0.571429,0.142857
2015-04-22,0.285714,0.571429,0.142857


In [57]:
print(p.state)

        15-Apr-15  16-Apr-15  17-Apr-15  20-Apr-15  22-Apr-15  Extrapolated  \
Symbol                                                                        
A        0.142857   0.142857   0.142857   0.142857   0.142857      0.142563   
B        0.142857   0.142857   0.142857   0.142857   0.142857      0.143357   
C        0.142857   0.142857   0.142857   0.142857   0.142857      0.142771   
D        0.142857   0.142857   0.142857   0.142857   0.142857      0.142100   
E        0.142857   0.142857   0.142857   0.142857   0.142857      0.142771   
F        0.142857   0.142857   0.142857   0.142857   0.142857      0.143669   
G        0.142857   0.142857   0.142857   0.142857   0.142857      0.142771   

             Gap  
Symbol            
A       0.000294  
B      -0.000500  
C       0.000086  
D       0.000757  
E       0.000086  
F      -0.000811  
G       0.000086  


## Rebalancing

Daily rebalancing is somewhat expensive. It is possible to "iron" the portfolio and rebalance either
* on a fixed grid in time
* with respect to threshold that shall not be exceeded

In [68]:
x = p.iron_time("3M")

In [69]:
# The pro
x.trading_days

[Timestamp('2013-01-31 00:00:00'),
 Timestamp('2013-04-30 00:00:00'),
 Timestamp('2013-07-31 00:00:00'),
 Timestamp('2013-10-31 00:00:00'),
 Timestamp('2014-01-31 00:00:00'),
 Timestamp('2014-04-30 00:00:00'),
 Timestamp('2014-07-31 00:00:00'),
 Timestamp('2014-10-31 00:00:00'),
 Timestamp('2015-01-30 00:00:00'),
 Timestamp('2015-04-22 00:00:00')]

In [70]:
# The portfolio is rebalanced at 
x.state

,30-Apr-14,31-Jul-14,31-Oct-14,30-Jan-15,22-Apr-15,Extrapolated,Gap
Symbol,,,,,,,
A,0.142857,0.142857,0.142857,0.142857,0.142857,0.126767,0.016090
B,0.142857,0.142857,0.142857,0.142857,0.142857,0.154675,-0.011818
C,0.142857,0.142857,0.142857,0.142857,0.142857,0.155355,-0.012498
D,0.142857,0.142857,0.142857,0.142857,0.142857,0.137544,0.005313
E,0.142857,0.142857,0.142857,0.142857,0.142857,0.142501,0.000356
F,0.142857,0.142857,0.142857,0.142857,0.142857,0.151378,-0.008520
G,0.142857,0.142857,0.142857,0.142857,0.142857,0.131780,0.011077
